<img src="https://gitlab.com/bivl2ab/academico/cursos-uis/ai/ai-uis-student/raw/master/imgs/banner_IA.png"  width="1000px" height="200px">

# **Taller 03:  Deep Learning**

## **Outline**

1. [Ejercicio 1.](#eje1)
2. [Ejercicio 2.](#eje2)

In [ ]:
#@title **Execute this cell**
#@markdown Please include your student id
import sys
import inspect

group_id = "ML-20221-Laconga" #@param {type:"string"}
assignment_id = group_id +'.taller_deep_learning'
student_id = "2022146" #@param {type:"string"}
"""
Put your student ID here

Example: student_id =  '2152145'
""" 

"\nPut your student ID here\n\nExample: student_id =  '2152145'\n"

In [ ]:
 #@title **Execute this cell**
#@markdown **UTILS**
#@markdown Please dont modify any line in this cell

import os
import json
import requests
from collections import namedtuple


Config = namedtuple('Config', ['server_name'])
config = Config(server_name='https://bivlabgrader.azurewebsites.net/api')


def check_solution_and_evaluate(assignment_id: str, student_func_str: str):

    # Set the endpoint and payload.
    payload = {
        'func_str': student_func_str,
        'assignment_id': assignment_id,
        'student_id': student_id
    }
    endpoint_url = config.server_name + '/CheckAndEvaluateSolution'
    # print(endpoint_url)

    # Make request to server with the data coming from the notebook.
    r = requests.post(endpoint_url, params=payload)
    pprint_json_response(r.json())
    return r


def pprint_json_response(response, indent=0):
    """Pretty print the response."""
    for key, value in response.items():
        print('\t' * indent + str(key.capitalize()))

        # If dictionary, do a recurrent call.
        if isinstance(value, dict):
            pprint_json_response(value, indent + 1)
        else:
            # Enumerate elements if list.
            if isinstance(value, list):
                if len(value) == 1:
                    print('\t' * (indent + 1) + str(value[0]))
                else:
                    for i, e in enumerate(value, start=1):
                        print('\t' * (indent + 1) + f'{i}. {e}')
            else:
                print('\t' * (indent + 1) + str(value))

In [ ]:
#@title **Imports**

import numpy as np
import tensorflow as tf
from tensorflow import keras

---
# **Ejercicio 1**  <a name="eje1"></a>
---

Vamos a desarrollar una DNN para resolver un problema de **clasificación**. Recuerde que en un problema de clasificación, la última capa debe tener una cierta cantidad de neuronas y una cierta función de activación. Implemente una función que:

- Utilice los datos de un dataset sintético _make_blobs_ que está incluido en [`sklearn`](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html#sklearn.datasets.make_blobs). <br>Utilice la información de tamaño de las caracteristicas para definir las características de la capa inicial.

- Mediante la función `train_test_split` tome el 80% de los datos para entrenar y la cantidad restante para testing, **desactivando** la opción shuffle.

- Cree una red neuronal que reciba en la capa de entrada **todas las caracteristicas del dataset** y además tenga las siguientes capas intermedias (ocultas):

 - Dos (2) capas de 512 neuronas
 - Dos (2) capas de 256 neuronas
 - Dos (2) capas de 128 neuronas
 - Dos (2) capas de 64 neuronas

   Y todas ellas con función de activación `relu`

- La red neuronal debe ser entrenada con un optimizador `SGD`, `epochs`=10,  `batch_size`=16, regla de minimización (loss): `'sparse_categorical_crossentropy'` y métrica de evaluación `'accuracy'`

- Devuelva el historial del entrenamiento `history`
- Devuelva el resultado del comando `evaluate` en la variable `scores`
- Devuelva el resultado del comando `predict` en la variable `yest`

In [ ]:
#@title **code student**
#@markdown No modifique las dos últimas lineas

def function_t06_1(data, data_y):
    #libraries
    import numpy as np
    import tensorflow as tf
    from tensorflow import keras
    from sklearn.model_selection import train_test_split
    from sklearn.datasets import make_blobs
    
    """Put your code here"""

    tf.random.set_seed(21)
    np.random.seed(21)

    X, y = make_blobs(n_samples= 100, n_features=2, centers=2)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    nclasses = 2

    dnn1 = tf.keras.Sequential([ 
        tf.keras.layers.Dense(512, activation='relu',input_shape=[X_train.shape[1]]),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(nclasses, activation='sigmoid')
    ])

    dnn1.summary()     

    y_train_ohe = tf.keras.utils.to_categorical(y_train, num_classes=nclasses)
    y_test_ohe = tf.keras.utils.to_categorical(y_test, num_classes=nclasses)

    dnn1.compile(optimizer=tf.keras.optimizers.SGD(), 
              loss='sparse_categorical_crossentropyy',
              metrics=['accuracy'])
    history = dnn1.fit(X_train,
          y_train_ohe,
          epochs=10,
          batch_size=16,
          validation_data=(X_test, y_test_ohe))
    scores = dnn1.evaluate(X_test, y_test)
    yest = dnn1.predict(X_test)
    
    return history, scores, yest

In [ ]:
#@title **send your answer**

import pandas as pd
import numpy as np
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
np.random.seed(21)

c = []
for i in range(6):
  c.append([np.random.randint(5)*np.random.rand(), np.random.randint(5)*np.random.rand()])

data, data_y = make_blobs(n_samples=2000, centers=c, n_features=13, random_state=21)
h, scores, ypred = function_t06_1(data, data_y)

plt.plot(h.history['loss'], label="loss")
plt.plot(h.history['accuracy'], label="accuracy")
plt.legend()
plt.show()

print("Check...")
student_func_str = inspect.getsource(function_t06_1)
r = check_solution_and_evaluate(assignment_id, student_func_str)

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_99 (Dense)            (None, 512)               1536      
                                                                 
 dense_100 (Dense)           (None, 512)               262656    
                                                                 
 dense_101 (Dense)           (None, 256)               131328    
                                                                 
 dense_102 (Dense)           (None, 256)               65792     
                                                                 
 dense_103 (Dense)           (None, 128)               32896     
                                                                 
 dense_104 (Dense)           (None, 128)               16512     
                                                                 
 dense_105 (Dense)           (None, 64)              

ValueError: ignored

---
# **Ejercicio 2**  <a name="eje2"></a>
---

Vamos a desarrollar una DNN para resolver un problema de **regresión**. La principal diferencia en estas arquitecturas esta en la capa de salida: solo debemos tener una neurona sin función de activación que nos permita dar una respuesta en terminos de la escala de los datos. Implemente una función que:

- Utilice los datos de _california_housing_ que están en [`sklearn`](https://scikit-learn.org/0.16/modules/generated/sklearn.datasets.fetch_california_housing.html#sklearn.datasets.fetch_california_housing). <br>Utilice la información de tamaño de las caracteristicas para definir las características de la capa inicial.

- Mediante la función `train_test_split` tome el 85% de los datos para entrenar y la cantidad restante para testing, **desactivando** la opción shuffle.

- Cree una red neuronal que reciba en la capa de entrada **todas las caracteristicas del dataset** y además tenga tres capas intermedias (ocultas) de 128, 64 y 32 neuronas, respectivamente. Las activaciones para las capas intermedias deben ser tangente hiperpolica `tanh`.

- La red neuronal debe ser entrenada con un optimizador `RMSprop`, `epochs`=5,  `batch_size`=128, regla de minimización (loss): `'mae'` y métrica de evaluación `'mse'`

- Devuelva el historial del entrenamiento `history`
- Devuelva el resultado del comando `predict` en la variable `yest`
- Devuelva el resultado del comando `evaluate` en la variable `scores`

In [ ]:
#@title **code student**
#@markdown No modifique las dos últimas lineas

def function_t06_2(data, data_y):
    #libraries
    import numpy as np
    import tensorflow as tf
    from tensorflow import keras
    from sklearn.model_selection import train_test_split
    from sklearn.datasets import california_housing

    """ Put your code here. """

    tf.random.set_seed(21)
    np.random.seed(21)

    X_train, X_test, y_train, y_test = None

    dnnr = None([
          None
          None
          None
          None
          None
      ])
    dnnr.None(None, 
                None,
                None)
    history = dnnr.None
    yest = dnnr.None
    scores = dnnr.None
    
    return history, yest, scores

SyntaxError: ignored

In [ ]:
#@title **send your answer**
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
import matplotlib.pyplot as plt

tf.random.set_seed(21)
np.random.seed(21)

data = fetch_california_housing()["data"]
data_y = fetch_california_housing()["target"]

h, yest, score = function_t06_2(data, data_y)

plt.plot(h.history['loss'], label="loss")
plt.plot(h.history['mse'], label="mse")
plt.legend()
plt.show()

print("Check...")
student_func_str = inspect.getsource(function_t06_2)
r = check_solution_and_evaluate(assignment_id, student_func_str)

NameError: ignored

---
<img src="https://gitlab.com/bivl2ab/academico/cursos-uis/ai/ai-uis-student/raw/master/imgs/bannerThanks.jpg" alt="Drawing" style="width:700px;"/>